In [6]:
from decimal import Decimal
from time import sleep

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm


def process_page(driver, elements):
    for idx, gl_i_wrap in enumerate(elements):
        title_element = gl_i_wrap.find_element(
            By.CSS_SELECTOR, "div.p-name.p-name-type-2 > a"
        )
        price_element = gl_i_wrap.find_element(
            By.CSS_SELECTOR, "div.p-price > strong > i"
        )
        title = title_element.text
        price = price_element.text
        yield title, price


target_pages = 60
results = {}

In [9]:
url = "https://search.jd.com/Search?keyword=4k%E7%94%B5%E8%A7%86&enc=utf-8&qrst=1&rt=1&stop=1&vt=2&suggest=2.def.0.V03--featuredump%2C&wq=4K&page=1&s=1&click=0"
options = Options()
# 無痕模式
options.add_argument("--incognito")
# 不載入圖片以加速
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)
driver = webdriver.Chrome(".web-driver/chromedriver.exe", options=options)
driver.minimize_window()
driver.delete_all_cookies()
driver.get(url)

for page in tqdm(range(target_pages), desc="page"):
    if page > 0:
        webdriver.ActionChains(driver).send_keys(webdriver.Keys.ARROW_RIGHT).perform()
    sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
    elements = driver.find_elements(By.CSS_SELECTOR, "#J_goodsList > ul > li > div")
    if not elements:
        print("stop")
        break
    for title, price in process_page(driver, elements):
        results[title] = price

print(f"total {len(results)} records")


C:\Users\moweb\AppData\Local\Temp\ipykernel_20556\2699422407.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(".web-driver/chromedriver.exe", options=options)
page: 100%|██████████| 60/60 [02:08<00:00,  2.14s/it]

total 1777 records


In [1]:
import pandas as pd


results = pd.read_csv("example.csv")
results = {row["title"]: row["price"] for idx, row in results.iterrows()}

In [51]:
import functools
import re

import pandas as pd


def match_patterns(s, /, *, patterns):
    for pattern in patterns:
        if g := pattern.search(s):
            return g[1]


def parse(results):
    df = pd.DataFrame({"title": results.keys(), "price": results.values()})

    # Parse brand
    brands = pd.read_csv("brands.csv")
    df["brand"] = df.title.apply(
        functools.partial(
            match_patterns,
            patterns=[re.compile(fr".*({s.sc}).*") for _, s in brands.iterrows()],
        )
    )

    # Parse size
    df["inch"] = df.title.apply(
        functools.partial(
            match_patterns,
            patterns=[
                re.compile(r"(\d+)\s*英?(?:寸|吋)"),
            ],
        )
    )

    # Parse resolution
    df["resolution"] = df.title.apply(
        functools.partial(
            match_patterns,
            patterns=[
                re.compile(r"(?:[^\da-z]+|\s)([1248]k)(?:[^\da-z]+|\s)", re.I),
            ],
        )
    )

    # Parse refresh rate
    df["frame_rate"] = df.title.apply(
        functools.partial(
            match_patterns,
            patterns=[
                re.compile(r"(?:[^\da-z]+|\s)(\d{2,}hz)(?:[^\da-z]+|\s)", re.I),
            ],
        )
    )

    return df


view = parse(results)
display(view)


,title,price,brand,inch,resolution,frame_rate
0,华为智慧屏 SE 65英寸 2+16GB 超薄电视 广色域鸿鹄画质 4K超高清智能液晶电视机...,2499.0,华为,65,4K,None
1,华为智慧屏 SE55 MEMC迅晰流畅 55英寸超薄全面屏 4K超高清智能电视 2GB+16...,1999.0,华为,55,4K,None
2,索尼（SONY）XR-65X91K 65英寸 全面屏4K HDR 专业游戏电视 PS5理想搭...,7499.0,索尼,65,4K,None
3,TCL雷鸟电视雀4 43英寸电视 4K超清超薄全面屏 全生态HDR10 AI远场语音 液晶智...,899.0,TCL,43,4K,None
4,小米电视EA55 2022款 55英寸 金属全面屏 远场语音 逐台校准4K超高清智能教育电视...,1299.0,小米,55,4K,None
...,...,...,...,...,...,...
1772,三星 （SAMSUNG）75英寸 4K高清Frame艺术画壁大屏智能电视QA75LS03BA...,18999.0,三星,75,4K,None
1773,长虹 43D5PF 43英寸全面屏教育电视 智能语音 4K解码 蓝光高清 手机投屏 平板液晶...,1799.0,None,43,4K,None
1774,新款65英寸4k液晶高清电视机32 42 50 55 60 75智能语音网络手机投屏防爆显示...,1888.0,None,65,4k,None
1775,长虹 75D4PS 75英寸超薄无边全面屏 2+16GB 智能语音 4K超高清 手机投屏 教...,3999.0,None,75,4K,None


In [ ]:
re.search(
    r"(?:[^\d\w]+|\s)([1248]k)(?:[^\d\w]+|\s)",
    # "华HUAWE为芯片WiFi高清机顶盒4K无线投屏魔盒网络电视盒子联通移动电信MWYMYNY 华为8G至臻版+回看+VIP+通 官方标配",
    "索尼（SONY）XR-65X91K 65英寸 全面屏4K HDR 专业游戏电视 PS5理想搭档 XR认知芯片 /120fps 65X91J升级款",
    re.I | re.U,
)[1]

In [52]:
# for i in range(40, 40 + 5):
#     print(view.loc[(pd.notna(view.brand) & pd.isna(view["inch"]))].iloc[i].title)
# display(view.loc[(pd.notna(view.brand) & pd.notna(view["inch"]))])
# display(view.loc[(pd.isna(view.brand))])
display(view.loc[(pd.notna(view.resolution))])
view.to_csv("view.csv")

,title,price,brand,inch,resolution,frame_rate
0,华为智慧屏 SE 65英寸 2+16GB 超薄电视 广色域鸿鹄画质 4K超高清智能液晶电视机...,2499.0,华为,65,4K,None
1,华为智慧屏 SE55 MEMC迅晰流畅 55英寸超薄全面屏 4K超高清智能电视 2GB+16...,1999.0,华为,55,4K,None
2,索尼（SONY）XR-65X91K 65英寸 全面屏4K HDR 专业游戏电视 PS5理想搭...,7499.0,索尼,65,4K,None
3,TCL雷鸟电视雀4 43英寸电视 4K超清超薄全面屏 全生态HDR10 AI远场语音 液晶智...,899.0,TCL,43,4K,None
4,小米电视EA55 2022款 55英寸 金属全面屏 远场语音 逐台校准4K超高清智能教育电视...,1299.0,小米,55,4K,None
...,...,...,...,...,...,...
1772,三星 （SAMSUNG）75英寸 4K高清Frame艺术画壁大屏智能电视QA75LS03BA...,18999.0,三星,75,4K,None
1773,长虹 43D5PF 43英寸全面屏教育电视 智能语音 4K解码 蓝光高清 手机投屏 平板液晶...,1799.0,None,43,4K,None
1774,新款65英寸4k液晶高清电视机32 42 50 55 60 75智能语音网络手机投屏防爆显示...,1888.0,None,65,4k,None
1775,长虹 75D4PS 75英寸超薄无边全面屏 2+16GB 智能语音 4K超高清 手机投屏 教...,3999.0,None,75,4K,None
